In [10]:
import os
import datetime
import numpy as np
import pandas as pd

import torch

from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader, TensorDataset

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

from datetime import datetime

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
device_dis = torch.device('cuda:0')
# device_gen = torch.device('cuda:0')

# 获取当前日期和时间
run_time = datetime.now()
model_kind = f'UNSW-NB15_Multi'

save_path = os.path.join(f'sampled_model_results', model_kind)
save_path = os.path.join(save_path, str(run_time))

os.makedirs(save_path, exist_ok=True)
os.makedirs(f'{save_path}', exist_ok=True)
os.makedirs(f'{save_path}/models', exist_ok=True)
os.makedirs(f'{save_path}/results', exist_ok=True)

#### 训练参数设置

In [3]:
batch_size = 256
train_epoches = 30
class_num = 10
last_epoch = -1
print_interval = 100

#### 加载数据集函数

In [4]:
def Load_UNSWNB15(path_train_data, path_test_data, binary_or_multi='multi'):

    categorical_columns = ['proto', 'service', 'state']
    
    classification = ['Normal', 'Fuzzers', 'Analysis', 'Backdoor', 'DoS', 
                    'Exploits', 'Generic', 'Reconnaissance', 'Shellcode', 'Worms']
    # 加载数据 train_num:125973, test_num:22544, total_data:148517
    data_train = pd.read_csv(path_train_data).copy()
    data_test = pd.read_csv(path_test_data).copy()
    total_data = pd.concat([data_train, data_test], axis=0) # 合并train和test
    total_data = total_data.drop(['id'], axis=1)
    train_num = len(data_train)
    test_num = len(data_test)

    # 特征
    features = total_data.iloc[:, :-2]     
    
    # 标签（以Binary/Multi形式加载Y的值）
    if binary_or_multi=='binary':    
        # 删除attack_cat列
        total_data = total_data.drop('attack_cat', axis=1)
        # 把labels转换为binary[0,1] 
        labels = total_data.iloc[:, -1]
    elif binary_or_multi=='multi':
        # 删除label列
        total_data = total_data.drop('label', axis=1)
        labels_class = total_data.iloc[:, -1]
        
        pdlist_class_dict = {}
        for index, data_class in enumerate(classification):
            pdlist_class_dict[data_class] = index
                
        labels = np.array([pdlist_class_dict[row] for row in np.array(labels_class)])
        
    # One-hot编码数据
    features = pd.get_dummies(features, columns=categorical_columns)
    
    # Min-Max标准化
    scaler = MinMaxScaler().fit(features)
    features = scaler.transform(features)

    # 凑形状，增加60列
    addition_number = 60
    addition_data = np.zeros((len(total_data), addition_number))
    features = np.concatenate((features, addition_data), axis=1)
    
    # X_train = features[:train_num][:, :, np.newaxis]
    # X_test = features[train_num:][:, :, np.newaxis]
    X_train = features[:train_num].astype(np.float32)
    X_test = features[train_num:].astype(np.float32)
    Y_train = labels[:train_num].astype(np.longlong)
    Y_test = labels[train_num:].astype(np.longlong)
    
        
    # X_train = torch.tensor(X_train, dtype=torch.float32)
    # X_test = torch.tensor(X_test, dtype=torch.float32)
    # Y_train = torch.LongTensor(Y_train)
    # Y_test = torch.LongTensor(Y_test)

    # 创建tensor
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    Y_train_tensor = torch.tensor(Y_train, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    Y_test_tensor = torch.tensor(Y_test, dtype=torch.long)

    # 创建DataLoader
    train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, test_loader

#### 加载模型

In [ ]:
sampled_path =  f'models/AMEGAN_UNSW-NB15_Multi-Class 2024-10-05 03:17:27.070724/Discriminator/Epoch10/Iter1/Wasserstein/[1, 4, 15, 0, 15, 0]-Accuracy:0.0004574093249557373 Latency:0.7487864077669902 .pt'
dis = torch.load(sampled_path).cuda()

dis = nn.DataParallel(dis)

#### 损失函数、优化器

In [7]:
criterion = nn.CrossEntropyLoss()

params_dis = [param for param in dis.parameters()]
optimizer_dis = Adam(params=params_dis, lr=1e-3)
scheduler_dis = CosineAnnealingLR(optimizer_dis, T_max=train_epoches, last_epoch=last_epoch)

#### 加载数据

In [8]:
path_train_data='datasets/UNSW-NB15/UNSW_NB15_training-set.csv'
path_test_data='datasets/UNSW-NB15/UNSW_NB15_testing-set.csv'

train_loader, test_loader = Load_UNSWNB15(path_train_data=path_train_data, 
                                          path_test_data=path_test_data, 
                                          binary_or_multi='multi')# 装载数据到loader里面)

#### 训练

In [9]:
best_accu = 0
best_epoch = 0
for epo in range(train_epoches):
    print(f'Training Epoch{epo+1}')
    
    dis.train()
    for index, (train_image, train_label) in enumerate(train_loader):
        train_image, train_label = train_image.view(train_image.shape[0], -1, train_image.shape[1]).cuda(non_blocking=True), train_label.to(torch.int64).cuda(non_blocking=True)
        train_label = torch.nn.functional.one_hot(train_label, class_num).to(torch.float32)
        optimizer_dis.zero_grad()
        
        outs_real = dis(train_image)
        
        dis_loss = criterion(outs_real, train_label)
        
        dis_loss.backward()
        optimizer_dis.step()
        scheduler_dis.step()
        
        correct = (torch.argmax(outs_real, dim=1) == torch.argmax(train_label, dim=1)).sum().item()
        if (index % print_interval) == 0 and index != 0:
            print(f'Epo{epo+1}/Iter{index+1}:Accu {correct/len(train_label)}')

    dis.eval()
        
    Y_pred = np.array([])
    Y_test = np.array([])
    for image, label in test_loader:
        Y_test = np.append(Y_test, label, axis=None)
        image, label = image.view(image.shape[0], -1, image.shape[1]).cuda(non_blocking=True), label.cuda(non_blocking=True)

        predicted = dis(image).view(label.shape[0], -1)
        predicted = torch.argmax(predicted, dim=1)
        
        Y_pred = np.append(Y_pred, predicted.cpu().numpy(), axis=None)
            
    accuracy_test = accuracy_score(Y_test, Y_pred)
    f1_test=f1_score(Y_test, Y_pred, average='macro')
    prs_test=precision_score(Y_test, Y_pred, average='weighted')
    recall_test = recall_score(Y_test, Y_pred, average='macro')
    
    if accuracy_test > best_accu:
        best_accu = accuracy_test
        best_epoch = epo
        best_pred = Y_pred
        ground_truth = Y_test

         #保存整个模型
        torch.save(dis, f'{save_path}/models/{model_kind}_Discriminator_{epo}_accu_{accuracy_test:.4f}.pth')
        np.save(f'{save_path}/models/best_pred_accu_{accuracy_test:.4f}.npy', best_pred)
        np.save(f'{save_path}/models/ground_truth_accu_{accuracy_test:.4f}.npy', ground_truth)
        
    print_lines = f'Epoch {epo+1}/{train_epoches}:\nAccuracy:{accuracy_test}\n'
    print_lines += f'Best Accuracy:{best_accu} - Epoch:{best_epoch+1}\n'
    print(print_lines)
    
    with open(f'{save_path}/results/{model_kind}_Multi.txt', 'a', encoding='utf-8') as file:
        # 将输出内容写入文件
        file.write(print_lines+'\n')

Training Epoch1


TypeError: forward() missing 3 required positional arguments: 'temperature', 'latency_to_accumulate', and 'supernet_or_sample'